In [1]:
1+1

2

In [3]:
import os
from dotenv import load_dotenv
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.tools import StructuredTool

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


def add(x: str, y: str) -> str:
    print(f"x:{x}, y:{y}")
    try:
        return str(float(x) + float(y))
    except Exception:
        return ValueError("계산 실패")


def lookup_population(city: str) -> str:
    print(f"city:{city}")

    population = {
        "서울": "900만",
        "부산": "300만",
    }
    return population.get(city.strip(), "정보 없음")


tools = [
    StructuredTool.from_function(
        name="Add",
        func=add,
        description="두 숫자를 더합니다.",
        args_schema={"x": str, "y": str},
    ),
    StructuredTool.from_function(
        name="PopulationLookup",
        func=lookup_population,
        description="도시명을 입력하면 인구를 반환합니다.",
        args_schema={"city": str},
    ),
]

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    verbose=True,
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

print("OpenAI Functions 실행:")
print(agent.run("5에 2를 더하고, 서울 인구도 알려줘"))





C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4112\1632668768.py:55: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4112\1632668768.py:63: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(agent.run("5에 2를 더하고, 서

OpenAI Functions 실행:


> Entering new AgentExecutor chain...


TypeError: Object of type type is not JSON serializable